# Capstone Final Project - Battle of the Neighborhoods

### Applied Data Science Capstone IBM/Coursera Course

## Table of Contents

 * [Introduction](#introduction)
 * [Data](#data)
 * [Methods](#methods)
 * [Analysis](#analysis)
 * [Results and Discussion](#results)
 * [Conclusion](#conclusion)

# Introduction <a name="introduction"></a>
The idea of this study is to clearly settle, once and for all, which city is best: San Francisco or New York City. I will frame this study by exploring which city gives "more bang for your buck". I will answer this question by comparing median rent prices of neighborhoods in each city (the buck), and venues that reside in these neighborhoods (the bang). This study will give clarity to individuals in both cities that are considering moving to the other. Additionally, it will inform current residents of San Francisco and New York City, of other neighborhoods within their own city they could potentially move to whether they want to pay less in rent or live near certain venues. Examples of this could be a family wanting to move to an area with more parks and playgrounds, a coffee enthusiast wanting to live near the most coffee shops as possible, or your average person looking to pay less rent in a neighborhood comparable to their current one.

# Data <a name="data"></a>


# Methods <a name="methods"></a>
## Cleaning Datasets

#### Import packages to clean neighborhood and transportation data.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Defining file paths so each csv can be read into a Pandas dataframe

In [2]:
#nyc rent/location
file_path2 = 'https://raw.githubusercontent.com/d-alvear/Coursera_Capstone/master/data/medianrent_1B_NYC.csv'

#### NYC and SF datasets only need columns to be renamed

In [3]:
df_nycRent = pd.read_csv(file_path2)
df_nycRent.head()
#dataframe looks good, just rename some of the columns
df_nycRent.rename(columns={' MedianRent ':'Rent', 'Lat':'Latitude', 'Long':'Longitude'}, inplace = True)

In [4]:
df_nycRent.head()
#df_nycRent.to_csv('nyc_rent.csv')
#df_sfRent.to_csv('sf_rent.csv')

,Neighborhood,Rent,Latitude,Longitude
0,West Harlem,"2,200.00",40.8189,-73.9522
1,Central Harlem,"2,025.00",40.8089,-73.9482
2,East Harlem,"2,150.00",40.7957,-73.9389
3,Upper West Side,"3,225.00",40.7870,-73.9754
4,Upper East Side,"2,850.00",40.7736,-73.9566


#### Now I will create a map of San Francisco with neighborhood markers

In [5]:
import folium
from folium import plugins

In [6]:
latitude = 40.7831 
longitude = -73.9712
# creating map of Manhattan using latitude and longitude values
map_nyc = folium.Map(location=[latitude, longitude], zoom_start=12.25)

# add markers to map
for lat, lng, neighborhood, rent in zip(df_nycRent['Latitude'], df_nycRent['Longitude'], df_nycRent['Neighborhood'], df_nycRent['Rent']):
    label = 'Neighborhood: {}, Median Rent: ${}'.format(neighborhood, rent)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc)

map_nyc

# Analysis <a name="analysis"></a>

In [7]:
CLIENT_ID = 'IW1DNBRJOUHECGXR0ST002ZPDJYOBCIRTROZZ5YJZFHCIP3Q' # your Foursquare ID
CLIENT_SECRET = 'TWMIYINYFXOLI0IGNPXXLCIRHFWLUNJ4VP2TJZXSJGUP2ZBN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IW1DNBRJOUHECGXR0ST002ZPDJYOBCIRTROZZ5YJZFHCIP3Q
CLIENT_SECRET:TWMIYINYFXOLI0IGNPXXLCIRHFWLUNJ4VP2TJZXSJGUP2ZBN


In [8]:
nyc_hoods = df_nycRent[['Neighborhood', 'Latitude', 'Longitude']]
nyc_hoods.head()

,Neighborhood,Latitude,Longitude
0,West Harlem,40.8189,-73.9522
1,Central Harlem,40.8089,-73.9482
2,East Harlem,40.7957,-73.9389
3,Upper West Side,40.7870,-73.9754
4,Upper East Side,40.7736,-73.9566


In [9]:
neighborhood_latitude = nyc_hoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = nyc_hoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = nyc_hoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of West Harlem are 40.8189, -73.9522.


In [10]:
# type your answer here
LIMIT = 50
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=IW1DNBRJOUHECGXR0ST002ZPDJYOBCIRTROZZ5YJZFHCIP3Q&client_secret=TWMIYINYFXOLI0IGNPXXLCIRHFWLUNJ4VP2TJZXSJGUP2ZBN&v=20180605&ll=40.8189,-73.9522&radius=500&limit=50'

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d2281bb342adf0038a552ee'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Manhattanville',
  'headerFullLocation': 'Manhattanville, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 40,
  'suggestedBounds': {'ne': {'lat': 40.8234000045, 'lng': -73.94626484631213},
   'sw': {'lat': 40.8143999955, 'lng': -73.95813515368788}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51573e61e4b06ee00f00307b',
       'name': 'Sushi Sushi',
       'location': {'address': '54 Tiemann Pl',
        'crossStreet': 'Broadway',
        'lat': 40.818595,
        'lng': -73.952721,
        'labeledLa

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Sushi Sushi,Sushi Restaurant,40.818595,-73.952721
1,Fumo,Italian Restaurant,40.821412,-73.950499
2,Chinelos,Mexican Restaurant,40.820419,-73.953826
3,St. Nicholas Park,Park,40.817885,-73.948438
4,La Nueva Flor de Broadway,Latin American Restaurant,40.821896,-73.953782


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

40 venues were returned by Foursquare.


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
nyc_venues = getNearbyVenues(names=nyc_hoods['Neighborhood'],
                                   latitudes=nyc_hoods['Latitude'],
                                   longitudes=nyc_hoods['Longitude']
                                  )


West Harlem
Central Harlem
East Harlem
Upper West Side
Upper East Side
Midtown
Hell's Kitchen
Garment District
Murry Hill
Chelsea
Gramercy Park
West Village
Greenwich Village
East Village
Soho
Lower East Side
Tribeca
Battery Park
Financial District


In [18]:
print(nyc_venues.shape)
nyc_venues.head()

(940, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,West Harlem,40.8189,-73.9522,Sushi Sushi,40.818595,-73.952721,Sushi Restaurant
1,West Harlem,40.8189,-73.9522,Fumo,40.821412,-73.950499,Italian Restaurant
2,West Harlem,40.8189,-73.9522,Chinelos,40.820419,-73.953826,Mexican Restaurant
3,West Harlem,40.8189,-73.9522,St. Nicholas Park,40.817885,-73.948438,Park
4,West Harlem,40.8189,-73.9522,La Nueva Flor de Broadway,40.821896,-73.953782,Latin American Restaurant


In [39]:
# one hot encoding
nyc_onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nyc_onehot['Neighborhood'] = nyc_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [nyc_onehot.columns[-1]] + list(nyc_onehot.columns[:-1])
nyc_onehot = nyc_onehot[fixed_columns]

nyc_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Beer Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Building,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Filipino Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health & Beauty Service,Historic Site,Hobby Shop,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Ski Shop,Smoke Shop,Social Club,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,West Harlem,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,West Harlem,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,West Harlem,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,West Harlem,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [41]:
nyc_grouped = nyc_onehot.groupby('Neighborhood').mean().reset_index()
nyc_grouped

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auditorium,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Beer Bar,Beer Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Building,Burger Joint,Burrito Place,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Filipino Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gay Bar,General College & University,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health & Beauty Service,Historic Site,Hobby Shop,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pharmacy,Piano Bar,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shanghai Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Ski Shop,Smoke Shop,Social Club,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Battery Park,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.02,0.00,0.00,0.00,0.000,0.000,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.02,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.000,0.08,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.04,0.00,0.00,0.000,0.04,0.00,0.00,0.000,0.00,0.00,0.000,0.000,0.00,0.02,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.000,0.04,0.00,0.00,0.000,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.000,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.020,0.00,0.02,0.000,0.00,0.00,0.020,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00,0.000,0.02,0.00,0.00,0.02,0.00,0.000,0.06,0.00,0.000,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.10,0.02,0.00,0.00,0.00,0.00,0.020,0.00,0.040,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.04,0.02,0.00,0.00,0.00,0.00,0.04,0.00,0.02,0.00,0.00,0.00,0.000,0.00,0.00,0.02,0.00,0.000,0.00,0.020,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00
1,Central Harlem,0.00,0.02,0.04,0.00,0.00,0.00,0.02,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.000,0.020,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.02,0.00,0.000,0.00,0.02,0.020,0.02,0.000,0.02,0.00,0.00,0.02,0.00,0.00,0.00,0.00,0.00,0.000,0.00,0.00,0.00,0.000,0.00,0.00,0.000,0.000,0.00,0.00,0.00,0.00

In [42]:
num_top_venues = 5

for hood in nyc_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = nyc_grouped[nyc_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park----
              venue  freq
0              Park  0.10
1       Coffee Shop  0.08
2     Memorial Site  0.06
3             Plaza  0.04
4  Department Store  0.04


----Central Harlem----
                             venue  freq
0  Southern / Soul Food Restaurant  0.08
1                French Restaurant  0.06
2                      Yoga Studio  0.04
3              American Restaurant  0.04
4                 Sushi Restaurant  0.04


----Chelsea----
                 venue  freq
0          Art Gallery  0.06
1   Italian Restaurant  0.06
2           Bagel Shop  0.04
3  American Restaurant  0.04
4                 Café  0.04


----East Harlem----
                venue  freq
0              Bakery  0.12
1  Mexican Restaurant  0.10
2         Pizza Place  0.10
3  Italian Restaurant  0.04
4          Taco Place  0.04


----East Village----
          venue  freq
0      Wine Bar  0.08
1   Coffee Shop  0.06
2           Bar  0.06
3        Garden  0.04
4  Dessert Shop  0.04


----Financial

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = nyc_grouped['Neighborhood']

for ind in np.arange(nyc_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyc_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Battery Park,Park,Coffee Shop,Memorial Site,Department Store,Cupcake Shop
1,Central Harlem,Southern / Soul Food Restaurant,French Restaurant,Yoga Studio,Arts & Crafts Store,Jazz Club
2,Chelsea,Italian Restaurant,Art Gallery,Theater,Bagel Shop,Coffee Shop
3,East Harlem,Bakery,Mexican Restaurant,Pizza Place,Latin American Restaurant,Spanish Restaurant
4,East Village,Wine Bar,Bar,Coffee Shop,Dessert Shop,Vegetarian / Vegan Restaurant
5,Financial District,Coffee Shop,Pizza Place,Sandwich Place,Hotel,Italian Restaurant
6,Garment District,Coffee Shop,Dance Studio,Italian Restaurant,Hotel,Theater
7,Gramercy Park,American Restaurant,Italian Restaurant,Café,Hotel,Restaurant
8,Greenwich Village,Italian Restaurant,Jazz Club,Cosmetics Shop,Pizza Place,Ice Cream Shop
9,Hell's Kitchen,Thai Restaurant,Wine Bar,Theater,Pizza Place,Mexican Restaurant


In [46]:
# set number of clusters
kclusters = 10

nyc_grouped_clustering = nyc_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 0, 7, 3, 0, 6, 0, 1, 2], dtype=int32)

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

nyc_merged = nyc_hoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
nyc_merged = nyc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

nyc_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,West Harlem,40.8189,-73.9522,9,Mexican Restaurant,Pizza Place,Sandwich Place,Park,Café
1,Central Harlem,40.8089,-73.9482,0,Southern / Soul Food Restaurant,French Restaurant,Yoga Studio,Arts & Crafts Store,Jazz Club
2,East Harlem,40.7957,-73.9389,7,Bakery,Mexican Restaurant,Pizza Place,Latin American Restaurant,Spanish Restaurant
3,Upper West Side,40.7870,-73.9754,8,Wine Bar,Italian Restaurant,Bookstore,Seafood Restaurant,American Restaurant
4,Upper East Side,40.7736,-73.9566,8,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center,Sushi Restaurant,Spanish Restaurant


In [49]:
nyc_merged

,Neighborhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,West Harlem,40.8189,-73.9522,9,Mexican Restaurant,Pizza Place,Sandwich Place,Park,Café
1,Central Harlem,40.8089,-73.9482,0,Southern / Soul Food Restaurant,French Restaurant,Yoga Studio,Arts & Crafts Store,Jazz Club
2,East Harlem,40.7957,-73.9389,7,Bakery,Mexican Restaurant,Pizza Place,Latin American Restaurant,Spanish Restaurant
3,Upper West Side,40.7870,-73.9754,8,Wine Bar,Italian Restaurant,Bookstore,Seafood Restaurant,American Restaurant
4,Upper East Side,40.7736,-73.9566,8,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center,Sushi Restaurant,Spanish Restaurant
5,Midtown,40.7549,-73.9840,0,Hotel,Theater,Coffee Shop,Bakery,Food Truck
6,Hell's Kitchen,40.7638,-73.9918,2,Thai Restaurant,Wine Bar,Theater,Pizza Place,Mexican Restaurant
7,Garment District,40.7547,-73.9916,6,Coffee Shop,Dance Studio,Italian Restaurant,Hotel,Theater
8,Murry Hill,40.7479,-73.9757,0,Japanese Restaurant,Hotel,Sushi Restaurant,American Restaurant,Burger Joint
9,Chelsea,40.7465,-74.0014,0,Italian Restaurant,Art Gallery,Theater,Bagel Shop,Coffee Shop


In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyc_merged['Latitude'], nyc_merged['Longitude'], nyc_merged['Neighborhood'], nyc_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Results and Discussion <a name="results"></a>

# Conclusion <a name="conclusion"></a>

### Cluster 0

In [52]:
nyc_merged.loc[nyc_merged['Cluster Label'] == 0, nyc_merged.columns[[0] + list(range(3, nyc_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Central Harlem,0,Southern / Soul Food Restaurant,French Restaurant,Yoga Studio,Arts & Crafts Store,Jazz Club
5,Midtown,0,Hotel,Theater,Coffee Shop,Bakery,Food Truck
8,Murry Hill,0,Japanese Restaurant,Hotel,Sushi Restaurant,American Restaurant,Burger Joint
9,Chelsea,0,Italian Restaurant,Art Gallery,Theater,Bagel Shop,Coffee Shop
10,Gramercy Park,0,American Restaurant,Italian Restaurant,Café,Hotel,Restaurant
16,Tribeca,0,Cocktail Bar,Gym / Fitness Center,American Restaurant,Sushi Restaurant,French Restaurant
18,Financial District,0,Coffee Shop,Pizza Place,Sandwich Place,Hotel,Italian Restaurant


### Cluster 1

In [53]:
nyc_merged.loc[nyc_merged['Cluster Label'] == 1, nyc_merged.columns[[0] + list(range(3, nyc_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,West Village,1,Cosmetics Shop,Italian Restaurant,Jazz Club,Coffee Shop,Bakery
12,Greenwich Village,1,Italian Restaurant,Jazz Club,Cosmetics Shop,Pizza Place,Ice Cream Shop


### Cluster 2

In [54]:
nyc_merged.loc[nyc_merged['Cluster Label'] == 2, nyc_merged.columns[[0] + list(range(3, nyc_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,Hell's Kitchen,2,Thai Restaurant,Wine Bar,Theater,Pizza Place,Mexican Restaurant


### Cluster 3

In [55]:
nyc_merged.loc[nyc_merged['Cluster Label'] == 3, nyc_merged.columns[[0] + list(range(3, nyc_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,East Village,3,Wine Bar,Bar,Coffee Shop,Dessert Shop,Vegetarian / Vegan Restaurant
15,Lower East Side,3,Coffee Shop,Cocktail Bar,Chinese Restaurant,Café,Art Gallery


### Cluster 4

In [56]:
nyc_merged.loc[nyc_merged['Cluster Label'] == 4, nyc_merged.columns[[0] + list(range(3, nyc_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,Battery Park,4,Park,Coffee Shop,Memorial Site,Department Store,Cupcake Shop


### Cluster 5

In [57]:
nyc_merged.loc[nyc_merged['Cluster Label'] == 5, nyc_merged.columns[[0] + list(range(3, nyc_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,Soho,5,Shoe Store,Boutique,Italian Restaurant,Men's Store,Clothing Store


### Cluster 6

In [58]:
nyc_merged.loc[nyc_merged['Cluster Label'] == 6, nyc_merged.columns[[0] + list(range(3, nyc_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,Garment District,6,Coffee Shop,Dance Studio,Italian Restaurant,Hotel,Theater


### Cluster 7

In [59]:
nyc_merged.loc[nyc_merged['Cluster Label'] == 7, nyc_merged.columns[[0] + list(range(3, nyc_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,East Harlem,7,Bakery,Mexican Restaurant,Pizza Place,Latin American Restaurant,Spanish Restaurant


### Cluster 8

In [60]:
nyc_merged.loc[nyc_merged['Cluster Label'] == 8, nyc_merged.columns[[0] + list(range(3, nyc_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
3,Upper West Side,8,Wine Bar,Italian Restaurant,Bookstore,Seafood Restaurant,American Restaurant
4,Upper East Side,8,Italian Restaurant,Seafood Restaurant,Gym / Fitness Center,Sushi Restaurant,Spanish Restaurant


### Cluster 9

In [61]:
nyc_merged.loc[nyc_merged['Cluster Label'] == 9, nyc_merged.columns[[0] + list(range(3, nyc_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,West Harlem,9,Mexican Restaurant,Pizza Place,Sandwich Place,Park,Café


### Cluster 10

In [62]:
nyc_merged.loc[nyc_merged['Cluster Label'] == 10, nyc_merged.columns[[0] + list(range(3, nyc_merged.shape[1]))]]

,Neighborhood,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
